In [2]:
import requests
import csv
from bs4 import BeautifulSoup

def download_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def extract_data(data):
    if data is None:
        print("Failed to download data.")
        return

    # Extract show information
    show_info = {
        'id': data.get('id', ''),
        'url': data.get('url', ''),
        'name': data.get('name', '')
    }

    # Extract episode information
    episodes = data.get('_embedded', {}).get('episodes', [])
    episode_data = []

    for episode in episodes:
        episode_info = {
            'season': episode.get('season', ''),
            'number': episode.get('number', ''),
            'type': episode.get('type', ''),
            'airdate': episode.get('airdate', ''),
            'airtime': episode.get('airtime', ''),
            'runtime': episode.get('runtime', ''),
            'average_rating': episode.get('rating', {}).get('average', ''),
            'summary': BeautifulSoup(episode.get('summary', ''), 'html.parser').get_text(),
            'medium_image': episode.get('image', {}).get('medium', ''),
            'original_image': episode.get('image', {}).get('original', '')
        }
        episode_data.append(episode_info)

    return show_info, episode_data

def save_to_csv(show_info, episode_data, filename):
    # Define the CSV headers
    show_headers = ['ID', 'URL', 'Name']
    episode_headers = [
        'Season', 'Episode Number', 'Type', 'Airdate', 'Airtime',
        'Runtime', 'Average Rating', 'Summary', 'Medium Image', 'Original Image'
    ]

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(show_headers)
        writer.writerow([show_info['id'], show_info['url'], show_info['name']])
        writer.writerow([])  # Add an empty row for separation
        writer.writerow(episode_headers)

        for episode in episode_data:
            writer.writerow([
                episode['season'],
                episode['number'],
                episode['type'],
                episode['airdate'],
                episode['airtime'],
                episode['runtime'],
                episode['average_rating'],
                episode['summary'],
                episode['medium_image'],
                episode['original_image']
            ])

    print(f"Data saved as {filename}")

# Provide the URL to download the data from
url = "http://api.tvmaze.com/singlesearch/shows?q=westworld&embed=episodes"

# Provide the filename for the CSV file
filename = "westworld_data.csv"

# Download the data
data = download_data(url)

# Extract the required data attributes
show_info, episode_data = extract_data(data)

# Save the data to a CSV file
save_to_csv(show_info, episode_data, filename)


Data saved as westworld_data.csv
